<a href="https://colab.research.google.com/github/nunsongi/quito-risk-mapping-python/blob/main/lluvia_estaciones.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Limpieza y estandarización de datasets de lluvia (Quito)

## Objetivo
Unificar y depurar las series de precipitación **horaria, diaria y mensual** de estaciones dentro del **DMQ (Quito)** para el periodo **2019 a 2025**, aplicando controles de calidad y completitud, y exportar **datasets limpios y consistentes** (diario y mensual) listos para análisis de inundaciones y para su integración con el notebook principal.

---

## Contenido
1. Importación de Librerías
2. Carga de datos
3. Filtro geográfico (solo Quito)
4. Normalización y limpieza
5. Manejo de series acumuladas (si aplica)
6. Agregaciones
7. Control de calidad (QA)
8. Unificación de granularidades
9. Consolidación por estación y por zona
10. Exportación




## 1. **Importación de Librerías**

In [4]:
# Importación de librerías

import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

## 2. **Carga de datos**

In [5]:
# Lluvias registro diario
lluvia_diario = pd.read_csv(r'/content/P09-Iñaquito_INAMHI_Precipitación-Diario.csv')

lluvia_diario.head()

,fecha,valor,completo_mediciones,completo_umbral
0,2014/07/01,0.0,100.0,100.0
1,2014/07/02,0.0,100.0,100.0
2,2014/07/03,0.0,100.0,100.0
3,2014/07/04,0.0,100.0,100.0
4,2014/07/05,0.0,100.0,100.0


In [7]:
# Mostrar las últimas filas

lluvia_diario.tail()

,fecha,valor,completo_mediciones,completo_umbral
4044,2025/07/27,NaN,NaN,NaN
4045,2025/07/28,NaN,NaN,NaN
4046,2025/07/29,NaN,NaN,NaN
4047,2025/07/30,NaN,NaN,NaN
4048,2025/07/31,NaN,NaN,NaN


In [8]:
df = lluvia_diario.copy()

# --- renombrar a nombres estándar
df = df.rename(columns={
    "fecha": "fecha",
    "valor": "p_mm_dia",                    # precipitación diaria en mm
    "completo_mediciones": "completo_med",
    "completo_umbral": "completo_umbral"
})

df.head()

,fecha,p_mm_dia,completo_med,completo_umbral
0,2014/07/01,0.0,100.0,100.0
1,2014/07/02,0.0,100.0,100.0
2,2014/07/03,0.0,100.0,100.0
3,2014/07/04,0.0,100.0,100.0
4,2014/07/05,0.0,100.0,100.0


In [9]:
# Mostrar la información del DataFrame
lluvia_diario.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4049 entries, 0 to 4048
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   fecha                4049 non-null   object 
 1   valor                3677 non-null   float64
 2   completo_mediciones  3776 non-null   float64
 3   completo_umbral      3776 non-null   float64
dtypes: float64(3), object(1)
memory usage: 126.7+ KB


In [10]:
# Comprobar datos duplicados

lluvia_diario.nunique()

,0
fecha,4049
valor,277
completo_mediciones,28
completo_umbral,18


In [11]:
# Calcular valores faltantes
lluvia_diario.isnull().sum()

,0
fecha,0
valor,372
completo_mediciones,273
completo_umbral,273


In [12]:
precipitaciones_diarias2 = pd.read_csv(r'/content/tableDownload (4).csv')

precipitaciones_diarias2.head()

,Fecha,RN
0,2019-11-06 23:50,173.2
1,2019-11-07 23:00,0.0
2,2019-11-08 23:00,0.0
3,2019-11-09 23:00,0.2
4,2019-11-10 23:00,0.4


In [13]:
# Lluvias horaria

precipitaciones_horaria = pd.read_csv(r'/content/P09-Iñaquito_INAMHI_Precipitación-Horario.csv')

precipitaciones_horaria.head()

,fecha,valor,completo_mediciones
0,2014/07/01 00:00:00,0.0,100.0
1,2014/07/01 01:00:00,0.0,100.0
2,2014/07/01 02:00:00,0.0,100.0
3,2014/07/01 03:00:00,0.0,100.0
4,2014/07/01 04:00:00,0.0,100.0


In [14]:
# Lluvia Mensual

precipitaciones_mensual = pd.read_csv(r'/content/P09-Iñaquito_INAMHI_Precipitación-Mensual.csv')

precipitaciones_mensual.head()

,fecha,valor,completo_mediciones,completo_umbral
0,2014/07,0.2,100.0,100.0
1,2014/08,2.9,100.0,100.0
2,2014/09,58.5,100.0,100.0
3,2014/10,110.9,100.0,100.0
4,2014/11,59.7,100.0,100.0


Renombrar a nombres estándar

In [15]:
# Manejo de duplicados

duplicados_fecha = df.duplicated(keep=False)
# df = df.drop_duplicates(subset=["fecha"]).sort_values("fecha")
conteo_duplicados_fecha = duplicados_fecha.sum()

print(f'conteo de fechas duplicadas: {conteo_duplicados_fecha}')

conteo de fechas duplicadas: 0
